Now I have made v3, which optimizes a few things and changes some function names to make the code more transparent.

In [1]:
import numpy as np
import sys, os

In [2]:
sys.path.insert(0, "../bin")
from Ladder_3_i0 import Ladder_3_i0 as L3i0
from Ladder_3_v2 import Ladder_3_v2 as L3v2 
from Ladder_3_v3 import Ladder_3_v3 as L3v3

In [3]:
obji0 = L3i0()
objv2 = L3v2()
objv3 = L3v3()

In [4]:
a12=0.15
b12=0.01
a23=0.1
b23=0.01
a12T=a12*10
a23T=a23*10
b12T=b12
b23T=b23
kon=1
koff=1
P=10
print(koff+b23T)
tf_conc =3
pars=np.array([a12,a23,b12,b23,a12T,a23T,b12T,b23T,kon,koff,P, P])

1.01


In [5]:
#original code
obj=obji0 
obj.setLaplacian_ladder_3(pars) 
obj.mult_laplacian_TF(3)
print(obj.get_FPT_stat(True))
%timeit obj.get_FPT_stat(True)

[[2.54363533]
 [0.65932379]]


In [6]:
#code where the transpose is done once and then used subsequently, plus some other changes like the possibility to not calculate cv of the FPT distribution
obj=objv2
obj.setLaplacian_ladder_3(pars) # super().setLaplacian_ladder_3(converted_pset)-> set_ladder_3_laplacian-> fills laplacian attribute
obj.mult_laplacian_TF(3)
print(obj.get_FPT_stat(True,True))
#%timeit obj.get_FPT_stat(True,True)

[[2.54363533]
 [0.65932379]]


In [9]:
#code where some further refinements are made and some function names are redefined for clarity
obj=objv3 
obj.setLaplacian(pars) # changed function name so it can be general
obj.mult_laplacian_TF(3)
print(obj.get_FPT_stat(True,True))
#%timeit obj.get_FPT_stat(True,True)

[[2.54363533]
 [0.65932379]]


In [10]:
(689-9)/(792+14)

0.8436724565756824

In [11]:
(689+9)/(792-14)

0.897172236503856

Note that it takes 10-15% less time just with these small changes.

In [12]:
#now do not to compute cv
obj=objv3
obj.setLaplacian(pars) # super().setLaplacian_ladder_3(converted_pset)-> set_ladder_3_laplacian-> fills laplacian attribute
obj.mult_laplacian_TF(3)
print(obj.get_FPT_stat(True,False))
%timeit obj.get_FPT_stat(True,False)

[[ 2.54363533]
 [-1.        ]]
340 µs ± 7.48 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Note that the calculation of the cv takes up half the time. So, if it is not needed, it should not be calculated.

In [13]:
p_log_range = 3                      # log10 parameter range
initial_tf_conc_range = 20           # initial TF concentration range
th_deltaR = 1                        # when th_deltaR = 1 the TF conc range used for the score computation is equal to the initial
n_sampled_params = 500               # number of sampled parameters x number of cpus 
n_points = 100                       # number of points on which computing the FPT score
low_mean, up_mean = 10**1, 10**4 # rejection range for the initial mean FPT

In [14]:
obj=obji0
TF_range = obj.triming([-initial_tf_conc_range, initial_tf_conc_range], 100, th_deltaR)

# compute score with rejection setted by the qt1 and qt3)
score = obj.simple_score([TF_range[0], TF_range[1]], n_points, low_mean, up_mean)
print(score)

[6.45002426e-1753 1.08945771e-0028]


In [15]:
obj=objv2
TF_range = obj.triming([-initial_tf_conc_range, initial_tf_conc_range], 100, th_deltaR)

# compute score with rejection setted by the qt1 and qt3)
score = obj.simple_score([TF_range[0], TF_range[1]], n_points, low_mean, up_mean)
print(score)

[6.45002426e-1753 0.00000000e+0000]


In [16]:
obj=objv3
TF_range = obj.triming([-initial_tf_conc_range, initial_tf_conc_range], 100, th_deltaR)

# compute score with rejection setted by the qt1 and qt3)
score = obj.simple_score([TF_range[0], TF_range[1]], n_points, low_mean, up_mean,computemeanscore=True, computecvscore=True,sqpar=0.01)
print(score)

[6.45002426e-1753 1.08945771e-0028]
